- [참고 data2vec audio:사용 안하기로함]: https://towhee.io/audio-embedding/data2vec
- [참고 data2vec text]: https://huggingface.co/docs/datasets/nlp_load
- [참고 data2vec load audio*]:https://huggingface.co/docs/datasets/audio_load
- [참고 data2vec embedding]: https://huggingface.co/docs/transformers/model_doc/data2vec#transformers.Data2VecAudioModel
- [참고 handling pytorch dataset]:https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel
- [참고 add data to dataset]: https://pypi.org/project/datasets/

# Embeddings(Text, Audio) Session 1 
- 각세션으로 확대시켜야 함

In [1]:
import torch
from transformers import AutoProcessor, Data2VecAudioModel
from datasets import load_dataset, Dataset, Audio, Features
from glob import glob
import numpy as np
import pandas as pd


/home/arplab/project/lou/multi_modal/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_name = 'Sess01_eval.csv'
df_annotation = pd.read_csv('org_KEMDy20/annotation/' + file_name, skiprows=1)
# df_annotation['Emotion'], df_annotation['Valence'], df_annotation['Arousal']
file_lst = df_annotation[' .1']
file_lst

0      Sess01_script01_User002M_001
1      Sess01_script01_User002M_002
2      Sess01_script01_User002M_003
3      Sess01_script01_User002M_004
4      Sess01_script01_User001F_001
                   ...             
306    Sess01_script06_User001F_016
307    Sess01_script06_User002M_041
308    Sess01_script06_User002M_042
309    Sess01_script06_User002M_043
310    Sess01_script06_User001F_017
Name:  .1, Length: 311, dtype: object

## Audio Embedding

In [3]:
wav_file_lst = []
for i in file_lst:
    wav_file_lst.append('./org_KEMDy20/Session01/' + i + '.wav')
wav_file_lst[:3]

['./org_KEMDy20/Session01/Sess01_script01_User002M_001.wav',
 './org_KEMDy20/Session01/Sess01_script01_User002M_002.wav',
 './org_KEMDy20/Session01/Sess01_script01_User002M_003.wav']

In [4]:
def get_file_names(file_lst):
    file_name_lst = []
    for i in file_lst:
        i = i.split('/')[-1]
        i = i.split('.')[-2]
        file_name_lst.append(i)
    return file_name_lst
    

In [5]:
import datasets

dataset = Dataset.from_dict({"wav": wav_file_lst }).cast_column("wav", Audio())
dataset[0]["wav"]

{'path': './org_KEMDy20/Session01/Sess01_script01_User002M_001.wav',
 'array': array([ 0.0526123 ,  0.05267334,  0.05197144, ..., -0.00387573,
        -0.00244141, -0.00137329], dtype=float32),
 'sampling_rate': 16000}

In [6]:
check_point_wav_model = 'facebook/data2vec-audio-base-960h'
processor = AutoProcessor.from_pretrained(check_point_wav_model)
model_wav_cuda = Data2VecAudioModel.from_pretrained(check_point_wav_model).cuda()
model_wav = Data2VecAudioModel.from_pretrained(check_point_wav_model)

Some weights of the model checkpoint at facebook/data2vec-audio-base-960h were not used when initializing Data2VecAudioModel: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Data2VecAudioModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Data2VecAudioModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/data2vec-audio-base-960h were not used when initializing Data2VecAudioModel: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Data2VecAudioModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from

In [7]:
dataset[0]['wav']['array']

array([ 0.0526123 ,  0.05267334,  0.05197144, ..., -0.00387573,
       -0.00244141, -0.00137329], dtype=float32)

In [8]:
sampling_rate = dataset['wav'][0]['sampling_rate']
sampling_rate

16000

In [9]:
for i in range(3):
    input = processor(dataset[i]['wav']['array'], sampling_rate = sampling_rate, return_tensors = 'pt', padding = True, max_length = 50)
    with torch.no_grad():
        output = model_wav(**input)
output['last_hidden_state'].shape

2023-03-29 18:24:41.684467: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 18:24:42.891267: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11/include:/usr/local/cuda-11/lib64::/usr/local/cuda/extras/CUPTI/lib64
2023-03-29 18:24:42.891382: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11/include:/usr/local/cuda

torch.Size([1, 410, 768])

### audio embedding method

In [10]:
# padding: https://huggingface.co/docs/transformers/pad_truncation
def wav_embeddings_func(wav_file_lst, processor, model):
    dataset = Dataset.from_dict({"wav": wav_file_lst }).cast_column("wav", Audio())
    wav_embeddings = []
    for i in range(len(dataset)): 
        inputs = processor(dataset[i]['wav']['array'], sampling_rate = sampling_rate, return_tensors = 'pt', padding = 'max_length', max_length= 16000, truncation = True)
        with torch.no_grad():
            output = model_wav(**inputs)
        wav_embeddings.append(output['last_hidden_state'])
    return wav_embeddings

## Text Embedding
- 참고: https://huggingface.co/docs/datasets/nlp_load
- 참고: https://huggingface.co/docs/transformers/v4.27.2/en/model_doc/data2vec#transformers.Data2VecTextModel
- 참고 한국어 pretrained model: https://huggingface.co/Junmai/KR-Data2VecText-v1

In [11]:
from transformers import PreTrainedTokenizerFast, BartModel
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import re


In [12]:
# text data loading
txt_file_lst = []
for i in file_lst:
    txt_file_lst.append('./org_KEMDy20/Session01/' + i + '.txt')
txt_file_lst[:3]

['./org_KEMDy20/Session01/Sess01_script01_User002M_001.txt',
 './org_KEMDy20/Session01/Sess01_script01_User002M_002.txt',
 './org_KEMDy20/Session01/Sess01_script01_User002M_003.txt']

In [13]:
# 참고: https://huggingface.co/gogamza/kobart-base-v2
# 참고: https://github.com/BM-K/Sentence-Embedding-Is-All-You-Need

# model = AutoModel.from_pretrained('gogamza/kobart-base-v2')  # or 'BM-K/KoSimCSE-bert-multitask'
# tokenizer = AutoTokenizer.from_pretrained('gogamza/kobart-base-v2')  # or 'BM-K/KoSimCSE-bert-multitask'
checkpoint = 'BM-K/KoSimCSE-bert-multitask'
model_txt = AutoModel.from_pretrained(checkpoint)  # or 'BM-K/KoSimCSE-bert-multitask'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)  # or 'BM-K/KoSimCSE-bert-multitask'


### text embedding method

In [14]:
def text_embeddings_func(txt_file_lst, tokenizer, model):
    sentences = []

    for i in txt_file_lst:    
        f = open(i, 'r')
        line = f.readline()
        line = re.sub('\n', '', line)
        line = re.sub('  ', ' ', line)
        line = line.rstrip().lstrip()
        sentences.append(line)
        f.close()
    start_index = np.random.randint(1, high = len(txt_file_lst)) - 10
    
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    txt_embeddings, _ = model_txt(**inputs, return_dict=False)
    return txt_embeddings
    

# 데이터셋 만들기

In [15]:
# # wav file embedding
# wav_embeddings = wav_embeddings_func(wav_file_lst, processor, model_wav)
# wav_embeddings[:3]

In [16]:
# # txt file embedding
# txt_embeddings = text_embeddings_func(txt_file_lst,tokenizer, model_txt)
# txt_embeddings[:3]

In [17]:
# file_name_lst = get_file_names(wav_file_lst)
# file_name_lst[:5]

In [18]:
# # annotation data 가져오기
# df_annotation = pd.read_csv('org_KEMDy20/annotation/' + file_name, skiprows=1)
# df_annotation.Arousal[:3], df_annotation.Emotion[:3], df_annotation.Valence[:3]

In [19]:
# print(len(txt_embeddings), len(wav_embeddings), len(file_name_lst))

In [20]:
def build_dataset(file_name_lst, 
                  text_embeddings, 
                  wav_embeddings, 
                  label_emotion,
                  label_arousal,
                  label_valence, 
                  session_name = 'Session01'):
    dataset = {'file_names': file_name_lst, 
         'text_embeddings': text_embeddings, 
         'wav_embeddings':wav_embeddings,
         'Emotion':label_emotion,
         'Arousal':label_arousal,
         'Valence':label_valence}
    return dataset

In [21]:
session_num_lst = []
annot_num_lst = []
for i in range(40):
    if i <= 8:
        session_num_lst.append('Session0' + str(i+1))
        annot_num_lst.append('Sess0' + str(i+1) + '_eval.csv')
    else:
        session_num_lst.append('Session' + str(i+1))
        annot_num_lst.append('Sess' + str(i+1) + '_eval.csv')
session_num_lst[-5:], annot_num_lst[-5:]

(['Session36', 'Session37', 'Session38', 'Session39', 'Session40'],
 ['Sess36_eval.csv',
  'Sess37_eval.csv',
  'Sess38_eval.csv',
  'Sess39_eval.csv',
  'Sess40_eval.csv'])

In [23]:
dataset = {}
for annot, session in zip(annot_num_lst[:3], session_num_lst[:3]):
    
    df_annotation = pd.read_csv('./org_KEMDy20/annotation/' + annot, skiprows=1)
    file_name_lst = df_annotation[' .1']
    
    txt_file_lst = []
    wav_file_lst = []
    for file_name in file_name_lst:
        txt_file_lst.append('./org_KEMDy20/' + session + '/' + file_name + '.txt')
        wav_file_lst.append('./org_KEMDy20/' + session + '/' + file_name + '.wav')
    
    txt_embeddings = text_embeddings_func(txt_file_lst, tokenizer, model_txt)
    wav_embeddings = wav_embeddings_func(wav_file_lst, processor, model_wav_cuda)
    
    dataset[session] = build_dataset(file_name_lst, 
                txt_embeddings, 
                wav_embeddings,  
                df_annotation.Emotion, 
                df_annotation.Arousal, 
                df_annotation.Valence)
    
    torch.cuda.empty_cache()
    print(f'{session}번째 작업이 완료 되었습니다.')

Session01번째 작업이 완료 되었습니다.
Session02번째 작업이 완료 되었습니다.
Session03번째 작업이 완료 되었습니다.


In [26]:
dataset.keys()

dict_keys(['Session01', 'Session02', 'Session03'])

In [27]:
# length test
for i in dataset['Session01'].keys():
    print(len(dataset['Session01'][i]))


311
311
311
311
311
311


# save dataset

In [24]:
import pickle

# save data set
with open('./model/data/lou_dataset_1_3.pkl', 'wb')as f:
    pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)

In [74]:
with open('./model/data/lou_dataset.pkl', 'rb') as f:
    dataset = pickle.load(f)
dataset['Session01'].keys()

dict_keys(['file_names', 'text_embeddings', 'wav_embeddings', 'Emotion', 'Arousal', 'Valence'])